### Customer Insights: Understand customer behavior, location trends, and engagement

### 1. List all States with more than 2 customers per state

In [ ]:
SELECT CustomerState, COUNT(*) Count_0f_Customers  --Count aggregate is necessary to complement the display request component
FROM Customer_T
GROUP BY CustomerState                             --Necessary after use of Aggregate
HAVING Count_Of_Customers >= 2;                    --Condition for the Group (of states)

### 2. Retrieve names and state of customers not in Florida, California, and Texas

In [ ]:
SELECT CustomerName, CustomerState
FROM Customer_T
WHERE CustomerState NOT IN ('CA', 'FL', 'TX')   --Conditional using comparison operator
ORDER BY CustomerName, CustomerState;

### 3. Find how many customers are there in each state

In [ ]:
SELECT CustomerState, COUNT(*) Count_0f_Customers 
FROM Customer_T             --Identical to Q1, although without a (group) condition
GROUP BY CustomerState;

### 4. List Customer ID and names for customers that did not place any order

In [ ]:
SELECT CustomerID, CustomerName  
FROM Customer_T             --Subquery in the words "that did not"
WHERE CustomerID NOT IN (   --...because we want to exclude these records 1st since they are
    SELECT CustomerID       --...are not part of the display request
    From Order_T
    )
;

### 5. For each Florida customer with less than three orders, display their names and how Many times each customer placed orders

In [ ]:
SELECT CustomerName, COUNT(OT.CustomerID) AS CountOfOrders --'For Each' ==> Group ; 'how many' ==> COUNT
FROM Customer_T CT LEFT OUTER JOIN Order_T OT              --Outer Joins allow us to include customers who
ON CT.CustomerID = OT.CustomerID                           --are in the customer table, but not the Order Table
WHERE CustomerState = 'FL'
GROUP BY CustomerName
HAVING CountOfOrders < 3; 

### 6. For each customer with less than 2 orders and outside California and Florida, display their names and how many times each customer placed orders

In [ ]:
SELECT CustomerName, COUNT(OrderID) AS CountOfOrders --Identical principles from Q5
FROM Customer_T CT LEFT OUTER JOIN Order_T OT
ON CT.CustomerID = OT.CustomerID
WHERE CustomerState NOT IN ('CA', 'FL')              --We use a comparison operator to filter rows 
GROUP BY CustomerName
HAVING CountOfOrders < 2;

### 7. List customer names and states that either ordered products with less than average price or they are located in any state except NY, NJ, TX (EXISTS Operator)

In [ ]:
SELECT DISTINCT CustomerName, CustomerState			 --'that either..' implies subquery 
FROM Customer_T CT
INNER JOIN Order_T OT ON CT.CustomerID = OT.CustomerID
INNER JOIN OrderLine_T OLT ON OT.OrderID = OLT.OrderID
WHERE EXISTS (									     --EXISTS allows connection from Product_T back to
	SELECT *									     --OrderLine_T via ProductID reference, w/o a JOIN
	FROM Product_T PT
	WHERE ProductID = OLT.ProductID
	AND ProductStandardPrice < (
		SELECT AVG(ProductStandardPrice)		     --Secondary condition 
		FROM Product_T
		)
	)
OR CustomerState NOT IN ('NY', 'NJ', 'TX'); -- alt "!=" logic 
--SQL does not allow us to place Aggregate functions in the WHERE clause
--Futhermore, we want to compare EACH record of products to to an AVG price
--Once we derive the AVG price 1st, then we can evaluate

### 8. Retrieve names and states of customers that have placed orders and are not located in California or Texas

In [ ]:
SELECT DISTINCT CustomerName, CustomerState    --DISTINCT, because we want to avoid multiple records of 
FROM Customer_T CT INNER JOIN Order_T OT       --same customer who has 1 > record in Order Table
ON CT.CustomerID = OT.CustomerID 
WHERE CustomerState NOT IN ('CA', 'FL', 'TX'); --Comparison operator to filter out records